In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from robopose.config import LOCAL_DATA_DIR
from IPython.display import display_html
import datetime
import torch

# Known Q, DREAM

In [ ]:
exp_id = 1804
robots = ['panda', 'kuka', 'baxter']
result_ids = [
    f'dream-{robot}-dream-all-models--{exp_id}' for robot in robots
]

rows = []
for result_id in result_ids:
    result_dir = LOCAL_DATA_DIR / 'results' / result_id
    print(result_id)
    ds_result_dirs = result_dir.glob('dataset=*')
    robot = result_id.split('-')[1]
    for ds_result_dir in ds_result_dirs:
        dataset = ds_result_dir.name.split('dataset=')[1]
        path = ds_result_dir / 'summary.pth.tar'
        summary = torch.load(path)
        for net in ('vgg_f', 'vgg_q', 'resnet_h'):
            key = f'known_joints/dream_net={robot}_dream_{net}'
            ADD = summary.get(f'{key}/dream_keypoints/ADD/AUC')
            if ADD is not None:
                rows.append(
                    dict(
                        dataset=dataset,
                        net=net,
                        ADD=ADD * 100,
                    )
                )
            
pd.options.display.float_format = "{:,.2f}".format
ds_names = [
        'dream.baxter.synt.dr.test',
        
        'dream.kuka.synt.dr.test',
        'dream.kuka.synt.photo.test',
        
        'dream.panda.synt.dr.test',
        'dream.panda.synt.photo.test',
        
        'dream.panda.real.azure',
        'dream.panda.real.kinect360',
        'dream.panda.real.realsense',
        
        'dream.panda.real.orb',
    ]
df = pd.DataFrame(rows).set_index('dataset')
for ds_name in ds_names:
    display_html(df.loc[[ds_name]].T)

# Known Q, Comparison with DREAM

In [ ]:
exp_id = 1804
robots = ['kuka', 'baxter', 'panda']
result_ids = [
    f'dream-{robot}-knownq--{exp_id}' for robot in robots
]

rows = []
for result_id in result_ids:
    print(result_id)
    result_dir = LOCAL_DATA_DIR / 'results' / result_id
    ds_result_dirs = result_dir.glob('dataset=*')
    for ds_result_dir in ds_result_dirs:
        dataset = ds_result_dir.name.split('dataset=')[1]
        path = ds_result_dir / 'summary.pth.tar'
        summary = torch.load(path)
        rows.append(
            dict(
                dataset=dataset,
#                 DREAM=summary['known_joints/dream/dream_keypoints/ADD/AUC'] * 100,
                ours=summary['full_image_detections/gt_joints/iteration=10/dream_keypoints/ADD/AUC'] * 100,
            )
        )

pd.options.display.float_format = "{:,.2f}".format
pd.DataFrame(rows).set_index('dataset').loc[
    [
        'dream.baxter.synt.dr.test',
        
        'dream.kuka.synt.dr.test',
        'dream.kuka.synt.photo.test',
        
        'dream.panda.synt.dr.test',
        'dream.panda.synt.photo.test',
        
        'dream.panda.real.azure',
        'dream.panda.real.kinect360',
        'dream.panda.real.realsense',
        
        'dream.panda.real.orb',
    ]
]

# Ablation Known Q

In [ ]:
from IPython.display import display_html
exp_id = 1804
infos = [
    dict(ref='link5'),
    dict(ref='link2'),
    dict(ref='link1'),
    dict(ref='link0'),
    dict(ref='link4'),
    dict(ref='link9'),
]


result_key = 'full_image_detections/gt_joints/iteration=10/dream_keypoints/ADD/AUC'
rows = []
for info in infos:
    result_id = f"dream-panda-orb-knownq-{info['ref']}--{exp_id}"
    print(result_id)
    result_dir = LOCAL_DATA_DIR / 'results' / result_id
    ds_result_dirs = result_dir.glob('dataset=*')
    for ds_result_dir in ds_result_dirs:
        dataset = ds_result_dir.name.split('dataset=')[1]
        summary = torch.load(ds_result_dir / 'summary.pth.tar')
        path = ds_result_dir / 'summary.pth.tar'
        summary = torch.load(path)
        row = dict(
            ref=info['ref'],
            dataset=dataset,
            ADD=summary[result_key] * 100
        )
        rows.append(row)
        
ds_names = [
    'dream.panda.real.orb',
]
df = pd.DataFrame(rows).set_index('dataset')
dfs = [df.loc[ds_name] for ds_name in ds_names]

pd.options.display.float_format = "{:,.2f}".format
for df_i in dfs:
    display_html(df_i)

# Results unknown joints

In [ ]:
# DREAM
import numpy as np
exp_id = 1804
robots = [
    'kuka', 
    'baxter',
    'panda'
]
result_ids = [
    f'dream-{robot}-unknownq--{exp_id}' for robot in robots
]

rows =  []
result_key = 'full_image_detections/unknown_joints/iteration=10/dream_keypoints/ADD/AUC'
for result_id in result_ids:
    result_dir = LOCAL_DATA_DIR / 'results' / result_id
    print(result_id)
    ds_result_dirs = result_dir.glob('dataset=*')
    for ds_result_dir in ds_result_dirs:
        dataset = ds_result_dir.name.split('dataset=')[1]
        path = ds_result_dir / 'summary.pth.tar'
        summary = torch.load(path)
        rows.append(
            dict(
                dataset=dataset,
                DREAM=summary['known_joints/dream/dream_keypoints/ADD/AUC']*100,
                ours=summary[result_key]*100,
            )
        )
display_html(pd.DataFrame(rows).set_index('dataset'))


print("CRAVES LAB")
result_key = 'full_image_detections/unknown_joints/iteration=10'
result_id = f'craves-lab-unknownq--{exp_id}'
print(result_id)
ds_result_dir = LOCAL_DATA_DIR / 'results' / result_id / f'dataset=craves.lab.real.test'
path = ds_result_dir / 'summary.pth.tar'
summary = torch.load(path)
for k in ('joint_error_mean', 'loc_error_xyz_abs_mean', 'loc_error_norm', 'rot_error_mean'):
    meter = 'craves'
    if 'loc' in k:
        mul = 100
    else:
        mul = 1
    value = summary[f'{result_key}/{meter}/{k}'] * mul
    value_top50 = summary[f'{result_key}/{meter}/{k}_top50'] * mul
    print(f'{k}: {value:.2f}/{value_top50:.2f}')
pck = summary[f'{result_key}/craves_keypoints/PCK@0.2']*100
print(f"PCK@0.2: {pck:.2f}")
    
    
print("")
print("CRAVES YT")
result_key = 'full_image_detections/unknown_joints/iteration=20/craves_keypoints/PCK@0.2'
focals = [500, 750, 1000, 1250, 1500, 1750, 2000]
rows = []
for focal in focals:
    result_id = f'craves-youtube-unknownq-focal={focal}--{exp_id}'
    print(result_id)
    ds_result_dir = LOCAL_DATA_DIR / 'results' / result_id / f'dataset=craves.youtube'
    path = ds_result_dir / 'summary.pth.tar'
    summary = torch.load(path)
    result_df = torch.load(ds_result_dir / 'error_dfs.pth.tar')
    mean_accuracy = result_df['full_image_detections/unknown_joints/iteration=20']['craves_keypoints'].keypoints_mean_accuracy
    for im_id, v in enumerate(mean_accuracy):
        rows.append(dict(im_id=im_id, focal=focal, accuracy=v.item()))
    print(f"f={focal}, {summary[result_key]*100:.2f}")
    
bests = []
idx = pd.DataFrame(rows).set_index(['im_id'])
for im_id in range(252):
    bests.append(np.max(idx.loc[im_id, 'accuracy']))
print("Best focal")
print(np.mean(bests))

# Ablation unknown joints 

In [ ]:
exp_id = 1804
import numpy as np
rows = []
result_key = 'full_image_detections/unknown_joints/iteration=10/dream_keypoints/ADD/AUC'
top_5_links = ['panda_link0', 'panda_link1', 'panda_link2', 'panda_link4', 'panda_link5']

infos = [
    dict(anchor='link5'),
    dict(anchor='link2'),
    dict(anchor='link1'),
    dict(anchor='link0'),
    dict(anchor='link4'),
    dict(anchor='link9'),
    
    dict(anchor='random_all'),
    dict(anchor='random_top5'),
    dict(anchor='random_top3'),
]

rows = []

ds_names = [
    'dream.panda.real.orb',
]
ds_name = ds_names[0]

for info in infos:
    result_id = f"dream-panda-orb-unknownq-{info['anchor']}--{exp_id}"
    print(result_id)
    ds_result_dir = LOCAL_DATA_DIR / 'results' / result_id / f'dataset={ds_name}'
    path = ds_result_dir / 'summary.pth.tar'
    summary = torch.load(path)
    row = info
    row['result_id'] = result_id
    row[ds_name] = summary[result_key] * 100
    row[ds_name] = f"{row[ds_name]:.2f}"
    row['anchor'] = info['anchor']
    rows.append(row)
    
pd.DataFrame(rows).loc[:, ['anchor', ds_name]]

# Benefits of iterative refinement

In [ ]:
exp_id = 1804
Ks_train = (1, 2, 3, 5)
Ks_test = (1, 2, 3, 5, 10)
result_ids = [
    (f'dream-panda-orb-train_K={K}--{exp_id}', K)  for K in Ks_train
]

rows = []
for (result_id, K_train) in result_ids:
    print(result_id)
    result_dir = LOCAL_DATA_DIR / 'results' / result_id
    ds_result_dirs = result_dir.glob('dataset=*')
    print(result_id)
    for ds_result_dir in ds_result_dirs:
        dataset = ds_result_dir.name.split('dataset=')[1]
        path = ds_result_dir / 'summary.pth.tar'
        summary = torch.load(path)
        for K in Ks_test:
            rows.append(
                dict(
                    K_train=K_train,
                    K_test=K,
                    ADD=summary[f'full_image_detections/unknown_joints/iteration={K}/dream_keypoints/ADD/AUC'] * 100,
                )
            )
df = pd.DataFrame(rows).set_index('K_train')
for K in Ks_train:
    display_html(df.loc[[K]].T)